In [2]:
import torch
import torch.nn as nn
from functorch import vmap
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F
from itertools import accumulate
import cvxpy as cp
import gurobipy
import numpy as np
from torch.autograd.functional import hessian
import torch
from torch.autograd import Function, Variable
from torch.nn import Module
from torch.nn.parameter import Parameter
import util
from collections import OrderedDict
import plotly.graph_objs as go
import random
import matplotlib.pyplot as plt
import pandas as pd
from baseline import MLP, RNNmodel
import keras
torch.set_default_tensor_type(torch.DoubleTensor)
torch.manual_seed(0)

In [9]:
from differentiable_DR_model import DRagent
dataset = np.load(f"dataset/data_N365_0.npz")
P1 = dataset["p"].max()
P2 = dataset["d"].max()
T = 24
layer = DRagent(P1, P2, T)
layer.load_state_dict(torch.load('result/model_gradient/model_0.pth'))
layer.eval()

DRagent(
  (costNN): ICNN2(
    (linear_z1): Linear(in_features=24, out_features=24, bias=False)
    (linear_y0): Linear(in_features=1, out_features=24, bias=True)
    (linear_y1): Linear(in_features=1, out_features=1, bias=True)
    (act): Softplus(beta=1, threshold=20)
  )
  (layer): OptLayer()
)

In [17]:
d = dataset["d"]
p = dataset["p"]
df_price = dataset['price']
N_train = 20
price_tensor = torch.from_numpy(df_price[10:10+N_train]).double()
d_tensor = torch.from_numpy(d[10:10+N_train]).double()
p_tensor = torch.from_numpy(p[10:10+N_train]).double()
y_tensor = tuple([d_tensor, p_tensor])
d_tensor2 = d_tensor.permute(1,0).unsqueeze(2)
c2, c1, _ = layer.approximate_cost2(d_tensor2, diff=True)
layer.E1.data, layer.E2.data, layer.eta.data, torch.mean(c2), torch.mean(c1)

(tensor([0.7490]),
 tensor([-0.7503]),
 tensor([0.9081]),
 tensor(16.4026),
 tensor(10.7039, grad_fn=<MeanBackward0>))